# Importer les data

In [49]:
#Importer les données
!curl https://topcs.blob.core.windows.net/public/FlightData.csv -o flightdata.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1552k  100 1552k    0     0  3276k      0 --:--:-- --:--:-- --:--:-- 3276k


In [50]:
#introduire les données dans un DataFrame Pandas
import pandas as pd
df= pd.read_csv('flightdata.csv')
df.head(5)

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,UNIQUE_CARRIER,TAIL_NUM,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,...,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DEL15,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,Unnamed: 25
0,2016,1,1,1,5,DL,N836DN,1399,10397,ATL,...,2143,2102.0,-41.0,0.0,0.0,0.0,338.0,295.0,2182.0,NaN
1,2016,1,1,1,5,DL,N964DN,1476,11433,DTW,...,1435,1439.0,4.0,0.0,0.0,0.0,110.0,115.0,528.0,NaN
2,2016,1,1,1,5,DL,N813DN,1597,10397,ATL,...,1215,1142.0,-33.0,0.0,0.0,0.0,335.0,300.0,2182.0,NaN
3,2016,1,1,1,5,DL,N587NW,1768,14747,SEA,...,1335,1345.0,10.0,0.0,0.0,0.0,196.0,205.0,1399.0,NaN
4,2016,1,1,1,5,DL,N836DN,1823,14747,SEA,...,607,615.0,8.0,0.0,0.0,0.0,247.0,259.0,1927.0,NaN


In [51]:
df.shape #donne les dimensions du df

(11231, 26)

In [52]:
df.isnull().values.any() #vérifie si le df contient des valeurs nulls

True

In [53]:
df.isnull().sum()  #permet de localiser les valeurs nuls

YEAR                       0
QUARTER                    0
MONTH                      0
DAY_OF_MONTH               0
DAY_OF_WEEK                0
UNIQUE_CARRIER             0
TAIL_NUM                   0
FL_NUM                     0
ORIGIN_AIRPORT_ID          0
ORIGIN                     0
DEST_AIRPORT_ID            0
DEST                       0
CRS_DEP_TIME               0
DEP_TIME                 107
DEP_DELAY                107
DEP_DEL15                107
CRS_ARR_TIME               0
ARR_TIME                 115
ARR_DELAY                188
ARR_DEL15                188
CANCELLED                  0
DIVERTED                   0
CRS_ELAPSED_TIME           0
ACTUAL_ELAPSED_TIME      188
DISTANCE                   0
Unnamed: 25            11231
dtype: int64

# Nettoyer les data

In [54]:
df=df.drop('Unnamed: 25',axis=1) #supprimer les colonne 
df.isnull().sum()

YEAR                     0
QUARTER                  0
MONTH                    0
DAY_OF_MONTH             0
DAY_OF_WEEK              0
UNIQUE_CARRIER           0
TAIL_NUM                 0
FL_NUM                   0
ORIGIN_AIRPORT_ID        0
ORIGIN                   0
DEST_AIRPORT_ID          0
DEST                     0
CRS_DEP_TIME             0
DEP_TIME               107
DEP_DELAY              107
DEP_DEL15              107
CRS_ARR_TIME             0
ARR_TIME               115
ARR_DELAY              188
ARR_DEL15              188
CANCELLED                0
DIVERTED                 0
CRS_ELAPSED_TIME         0
ACTUAL_ELAPSED_TIME    188
DISTANCE                 0
dtype: int64

In [55]:
df = df[["MONTH", "DAY_OF_MONTH", "DAY_OF_WEEK", "ORIGIN", "DEST", "CRS_DEP_TIME", "ARR_DEL15"]] #restructuer mon df aka supprimer toutes les colonnes à part celles citées
df.isnull().sum()

MONTH             0
DAY_OF_MONTH      0
DAY_OF_WEEK       0
ORIGIN            0
DEST              0
CRS_DEP_TIME      0
ARR_DEL15       188
dtype: int64

In [56]:
df[df.isnull().values.any(axis=1)].head #montrer les lignes contenant des cellules vides

<bound method NDFrame.head of        MONTH  DAY_OF_MONTH  DAY_OF_WEEK ORIGIN DEST  CRS_DEP_TIME  ARR_DEL15
177        1             9            6    MSP  SEA           701        NaN
179        1            10            7    MSP  DTW          1348        NaN
184        1            10            7    MSP  DTW           625        NaN
210        1            10            7    DTW  MSP          1200        NaN
478        1            22            5    SEA  JFK          2305        NaN
481        1            22            5    ATL  JFK          1745        NaN
491        1            22            5    MSP  JFK          1755        NaN
499        1            23            6    JFK  ATL          1325        NaN
500        1            23            6    JFK  ATL          1540        NaN
501        1            23            6    JFK  SEA           740        NaN
502        1            23            6    JFK  ATL           610        NaN
503        1            23            6    JFK

In [57]:
df=df.fillna({'ARR_DEL15':1}) # remplire les cases vides de APR_DEL15 par 1
df.iloc[175:185]  #afficher les lignes entre 175 et 185

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN,DEST,CRS_DEP_TIME,ARR_DEL15
175,1,9,6,DTW,ATL,1205,1.0
176,1,9,6,JFK,DTW,1845,0.0
177,1,9,6,MSP,SEA,701,1.0
178,1,9,6,DTW,JFK,1527,0.0
179,1,10,7,MSP,DTW,1348,1.0
180,1,10,7,DTW,MSP,1540,0.0
181,1,10,7,JFK,ATL,1325,0.0
182,1,10,7,JFK,ATL,610,0.0
183,1,10,7,JFK,SEA,1615,0.0
184,1,10,7,MSP,DTW,625,1.0


# Préparer les données pour le modèle ML

In [58]:
import math
#normaliser les données
for index, row in df.iterrows():
    df.loc[index, 'CRS_DEP_TIME'] = math.floor(row['CRS_DEP_TIME'] / 100) #diviser et arrondire pour affiner l'intervalle
df.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN,DEST,CRS_DEP_TIME,ARR_DEL15
0,1,1,5,ATL,SEA,19,0.0
1,1,1,5,DTW,MSP,13,0.0
2,1,1,5,ATL,SEA,9,0.0
3,1,1,5,SEA,MSP,8,0.0
4,1,1,5,SEA,DTW,23,0.0


In [59]:
df=pd.get_dummies(df, columns=['ORIGIN','DEST'])  #subdiviser les colonnes ORIGIN et DEST 
df.head


<bound method NDFrame.head of        MONTH  DAY_OF_MONTH  DAY_OF_WEEK  CRS_DEP_TIME  ARR_DEL15  ORIGIN_ATL  \
0          1             1            5            19        0.0           1   
1          1             1            5            13        0.0           0   
2          1             1            5             9        0.0           1   
3          1             1            5             8        0.0           0   
4          1             1            5            23        0.0           0   
5          1             1            5            11        0.0           0   
6          1             2            6            17        0.0           1   
7          1             2            6            17        0.0           0   
8          1             2            6            11        0.0           0   
9          1             2            6            14        0.0           0   
10         1             2            6             8        0.0           1   
11        